#Funciones Lambda

Para entender las funciones lambda tenemos que tener claro primero las funciones normales: 

In [0]:
# creamos una función de sumar dos números y nos devuelve su suma
def sumar(a,b):
    suma = a + b
    return suma

In [0]:
# probamos la función con los argumentos dados
sumar(2,8)

Out[4]: 10

#### En RDDs podemos cambiar la estructura utilizada en python por otras que nos ayudan más para trabajar en Big Data:

In [0]:
# podemos usar sc.parallelize para crear una lista y la llamamos con .collect()
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
numeros.collect()

##TRANSFORMACIONES

In [0]:
# establecemos una lista
numeros = sc.parallelize([1,2,3,4,5])
# utilizando la transformación map con la función lambda que asignemos hacemos que cada elemento de la lista se multiplique por 3
num3 = numeros.map(lambda elemento: 3*elemento)
# mostramos por pantalla
num3.collect()


Out[10]: [3, 6, 9, 12, 15]

In [0]:
# creamos una lista en palabras que está llena de strings escritos de forma diferente
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])
# ahora con map y la función lambda hacemos que cada elemento de la lista se convierta en minúsculas y lo guardamos en otra variable
pal_minus = palabras.map(lambda elemento: elemento.lower())
# mostramos por pantalla
pal_minus.collect()

Out[3]: ['hola', 'que', 'tal', 'bien']

In [0]:
# establecemos una lista
numeros = sc.parallelize([1,2,3,4,5])
# ahora con la función filter y lambda podemos filtrar los elementos de la lista estableciéndoles una condición, como que sean pares (ya que su resto entre 2 es 0)
pares_rdd = numeros.filter(lambda elemento: elemento%2 == 0)
# mostamos por pantalla
pares_rdd.collect()

Out[2]: [2, 4]

In [0]:
# tenemos una lista de errores y warnings
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])
# queremos filtrar los errores, que son los elementos que empiezan con E, por eso usamos filter con la función lambda temporal que queremos filtrar
errors = log.filter(lambda elemento: elemento[0]=='E')
# mostamos por pantalla
errors.collect()

Out[4]: ['E: e21', 'E: e45']

#####Recuperar del log anterior los eventos de la linea/celda 12 (cadenas que terminen en <12>):

In [0]:
# creamos una variable para almacenar los eventos que queremos filtrar, aplicamos filter() con la función lambda que deseamos, este caso con formato slicing para cada elemento que acabe con 12
events = log.filter(lambda elemento: elemento[4:6]=='12')
# mostamos por pantalla
events.collect()

Out[16]: ['W: w12']

In [0]:
# EXTRA: aquí con esta función vemos qué variables se han declarado
%whos

Variable    Type            Data/Info
-------------------------------------
errors      PipelinedRDD    PythonRDD[6] at wrapper a<...>mmand-3540625238406604>:6
log         RDD             ParallelCollectionRDD[5] <...>am at PythonRDD.scala:435
numeros     RDD             ParallelCollectionRDD[1] <...>am at PythonRDD.scala:435
pal_minus   PipelinedRDD    PythonRDD[4] at wrapper a<...>mmand-3540625238406601>:6
palabras    RDD             ParallelCollectionRDD[3] <...>am at PythonRDD.scala:435
pares_rdd   PipelinedRDD    PythonRDD[2] at wrapper a<...>mmand-3540625238406603>:6


In [0]:
numeros = sc.parallelize([1,2,3,4,5])

rdd = numeros.flatMap(lambda elemento : [elemento, 10*elemento]) # esto nos da una salida toda plana, en una lista entera sin separaciones

rdd2 = numeros.map(lambda elemento : [elemento, 10*elemento]) # esto da una salida que separa en listas con las condiciones en una lista grande


rdd.collect()

Out[28]: [[1, 10], [2, 20], [3, 30], [4, 40], [5, 50]]

In [0]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras_map = lineas.map(lambda elemento: elemento.split())

palabras_flat = lineas.flatMap(lambda elemento: elemento.split())
# aquí vamos a ver mejor la diferencia entre map y flatmap 
print('RDD ORIGINAL: ', lineas.collect())
print('RDD resultado MAP: ', palabras_map. collect())
print('RDD resultado FLATMAP: ', palabras_flat.collect())


RDD ORIGINAL:  ['', 'a', 'a b', 'a b c']
RDD resultado MAP:  [[], ['a'], ['a', 'b'], ['a', 'b', 'c']]
RDD resultado FLATMAP:  ['a', 'a', 'b', 'a', 'b', 'c']


In [0]:
mensaje = "el quijote mola mazo manin"
mensaje.split("m") # esto separa las palabras por la cadena que queremos que las separe
mensaje.split()

Out[34]: ['el', 'quijote', 'mola', 'mazo', 'manin']

In [0]:
numeros = sc.parallelize([1,1,2,2,5])

unicos = numeros.distinct()
unicos.collect()


Out[43]: [1, 2, 5]

In [0]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

muestra = numeros.sample(False, 0.1) # True CON reemplazamiento, False SIN reemplazamiento /// el número es un número entre 0 y 1 en el que te da unos valores aproximados a los que se pone, por ejemplo 0.5 te saldrían, de una lista de 10, aproximadamente una lista de 5 valores. En listas pequeñas no tiene sentido pero en 1 millón de datos se cogen la mitad de samples de forma aleatoria y da igual cuántos son, mientras se coja aproximadamente

muestra.collect()

Out[88]: []

In [0]:
pares = sc.parallelize([2,4,6,8,10])
impares = sc.parallelize([1,3,5,7,9])

numeros = pares.union(impares)

numeros.collect()

Out[42]: [2, 4, 6, 8, 10, 1, 3, 5, 7, 9]

In [0]:
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])
log.setName("RDD ORIGINAL (partida)")

info = log.filter(lambda elemento: elemento[0]=='I')
info.setName("RDD resultado filtrado INFO")

error = log.filter(lambda elemento: elemento[0]=='E')
error.setName("RDD resultado filtrado ERROR")

inferr = info.union(error)
inferr.setName("RDD resultado UNION")

inferr.collect()


Out[64]: ['I: i11', 'I: i11', 'E: e21', 'E: e45']

###ACCIONES

In [0]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

pares = numeros.filter(lambda elemento: elemento%2==0)

pares.count()


Out[65]: 5

In [0]:
numeros = sc.parallelize([1,2,3,4,5])

numeros.reduce(lambda elem1,elem2: elem1+elem2) # esto suma todos los números, uno tras otro, de la variable numeros. reduce los parámetros de entrada, que son 2, en 1


Out[66]: 15

In [0]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print(pal_minus.reduce(lambda elem1,elem2: elem1 + " " + elem2))


hola que tal bien


In [0]:
palabras2 = sc.parallelize(['Como', 'te', 'encuentras', 'hoy'])


In [0]:
def cadena_larga(elem1, elem2):
 
    if len(elem1) >= len(elem2):
        return elem1
    else:
        return elem2


In [0]:
palabras2.reduce(cadena_larga)


Out[50]: 'encuentras'

In [0]:
numeros = sc.parallelize([5,3,2,1,4])
numeros.take(3)

Out[73]: [5, 3, 2]

In [0]:
numeros = sc.parallelize([3,2,4,5,1])

print(numeros.takeOrdered(3))


[1, 2, 3]


In [0]:
numeros = sc.parallelize([3,2,1,4,5])

print(numeros.takeOrdered(3, lambda elem: -elem)) # esto devuelve la lista reordenada dada al revés. lo cambia internamente ordenado y lo cambia también cambiado de signo


[5, 4, 3]


In [0]:
empleados = sc.parallelize([("Juan Carlos", 1700, 32, True), 
                            ("Marco Antonio", 3600, 26, True), 
                            ("Alicia Prieto", 600, 18, False)])
# nombre, sueldo, edad, activo (Sí/no)
empleados.collect()  # para mostrarlos por pantalla

Out[76]: [('Juan Carlos', 1700, 32, True),
 ('Marco Antonio', 3600, 26, True),
 ('Alicia Prieto', 600, 18, False)]

In [0]:
activos = empleados.filter(lambda elemento: elemento[3] is True) # mejor utilizar is True que == True, mejor práctica de programacion

activos.collect()

Out[83]: [('Juan Carlos', 1700, 32, True), ('Marco Antonio', 3600, 26, True)]